---
## I. Tris
La complexité optimale d'un algorithme de tri par comparaisons est $O(n\log n)$. La démonstration repose sur la représentation d'un algorithme de tri par comparaisons sous la forme d'un arbre binaire. Chaque noeud de l'arbre représente une comparaison entre deux éléments du tableau. La hauteur de l'arbre est donc le nombre de comparaisons nécessaires pour trier le tableau dans le pire cas. Le nombre de feuilles de l'arbre est le nombre de permutations possibles du tableau. Comme il y a $n!$ permutations possibles, la hauteur de l'arbre est donc au moins $\log_2(n!)$. En utilisant la formule de Stirling, on obtient que la hauteur de l'arbre est au moins $n\log_2(n) - O(n)$, ce qui prouve que la complexité d'un algorithme de tri par comparaisons est au moins $O(n\log n)$.

Pour aller plus vite, on peut affirmer que l'on peut distinguer $2^h$ cas avec $h$ comparaisons, et on veut donc $2^h \geq n!$. La suite est identique.

### I. 1. Tri fusion

Tri en complexité optimale le plus simple à implémenter.

**Complexité :** $O(n\log n)$ dans tous les cas

In [ ]:
let rec merge l1 l2 = 
  match (l1, l2) with
  | ([], _) -> l2
  | (_, []) -> l1
  | (h1::t1, h2::t2) -> if h1 < h2 then h1 :: merge t1 l2 else h2 :: merge l1 t2
;;

let rec split x y z =
  match x with
  | [] -> (y, z)
  | e :: x' -> split x' (e::z) y
;;

let rec tri_fusion l = 
  match l with
  | ([] | _::[]) -> l
  | _ -> let l1, l2 = split l [] [] in merge (tri_fusion l1) (tri_fusion l2)
;;

### I. 2. Tri rapide (implémentation mauvaise en pratique)

**Complexité :** $O(n^2)$ dans le pire cas, $O(n\log n)$ en moyenne

In [ ]:
(* Mauvaise implémentation en pratique, à la fois en temps (à cause de @) et en mémoire (structure persistante) *)
let rec quicksort l =
  match l with
  | ([] | _::[]) -> l
  | a::l' -> let inf = List.filter (fun x -> x<a) l' 
    and sup = List.filter (fun x -> x>a) l'
    in quicksort inf @ (a :: quicksort sup)

### I. 3. Tri à bulle
Tri en place, stable, en $O(n^2)$. Tri de complexité quelconque le plus simple à implémenter.

In [ ]:
let swap t i j =
  let mem = t.(i) in
  t.(i) <- t.(j) ;
  t.(j) <- mem
;;

let tri_bulle t =
  let n = Array.length t in
  for i = n-1 downto 1 do
    for j = 0 to i-1 do
      if t.(j) > t.(j+1) then swap t j (j+1)
    done
  done 
;;

---
## II. Arbres binaires de recherche

Un arbre binaire de recherche est un arbre binaire étiqueté par un ensemble totalement ordonné, tel que pour tout noeud, les étiquettes des noeuds du sous-arbre gauche sont inférieures à l'étiquette du noeud, et les étiquettes des noeuds du sous-arbre droit sont supérieures à l'étiquette du noeud.

Cette structure sert notamment à implémenter des dictionnaires. 


### II. 1. Fonctions de base

**Complexité :** $O(h)$, où $h$ est la hauteur de l'arbre. On a $h = O(\log n)$ en moyenne, mais $h = O(n)$ dans le pire cas (peigne).

In [ ]:
type 'a tree =
  | V
  | N of 'a tree * 'a * 'a tree
;;

let rec recherche arbre el =
  match arbre with
  | V ->  false
  | N(g, a, d) -> if el = a then true 
                  else if el < a then recherche g el 
                  else recherche d el
;;

let rec ajout arbre el =
  match arbre with
  | V -> N(V, el, V)
  | N(g, a, d) -> if el < a then N(ajout g el, a, d) else N(g, a, ajout d el)
;;

(* Première méthode de suppression : on cherche le plus grand élément y du SAG,
on le met à la place de l'élément à supprimer et on supprimer récursivement y du SAG *)
let rec maxi arbre = (* cherche récursivement le maximum *)
  match arbre with
  | V -> None
  | N(g, a, V) -> Some a
  | N(g, a, d) -> maxi d
;;

let rec suppression arbre el =
  match arbre with
  | V -> V
  | N(V, a, d) when a = el -> d
  | N(g, a, V) when a = el -> g
  | N(g, a, d) -> if el < a then N(suppression g el, a, d)
                  else if el > a then N(g, a, suppression d el)
                  else match maxi g with (* on cherche le maximum *)
                  | None -> failwith "erreur"
                  | Some y -> N(suppression g y, y, d) (* on le met à la racine et on le supprime de g *)
;;

(* Seconde méthode de suppression : même idée mais en faisant un seul parcours *)
let rec suppression_bis arbre el =
  let rec extract_max arbre = (* retourne (le SAG sans son maximum, le maximum) *)
    match arbre with
    | V -> failwith "erreur"
    | N(g, a, V) -> (g, a) (* on a trouvé le maximum et son SAG *)
    | N(g, a, d) -> let (d', v) = extract_max d in (N(g, a, d'), v) (* on cherche la maximum à droite *)
  in
  match arbre with
  | V -> V
  | N(V, a, d) when a = el -> d
  | N(g, a, V) when a = el -> g
  | N(g, a, d) -> if el < a then N(suppression_bis g el, a, d)
                  else if el > a then N(g, a, suppression_bis d el)
                  else (* a = el *)
                     let g', y = extract_max g in N(g', y, d) (* on remplace le SAG par le SAG sans son maximum *)
;;

In [ ]:
let rec partition e = function (* partitionne un arbre selon les valeurs plus petites et plus grandes que e *)
  | V - > V, V
  | N(g, x, d) when x < e -> (* on partitionne à droite *)
    let a1, a2 = partition e d in (N(g, x, a1), a2)
  | N(g, x, d) -> (* on partitionne à gauche *)
    let a1, a2 = partition e g in (a1, N(a2, x, d))
;;

In [ ]:
(* Fusionne deux arbres binaires de recherche *)
let rec fusion a1 a2 = 
  match (a1, a2) with
  | _, V -> a1
  | V, _  -> a2
  | _, N(g2, x, d2) -> (* On partitionne a1 selon la racine de a2, et on fusionne les sous-arbres découpés *)
    let g1, d1 = partition x a1 in N(fusion g1 g2, x, fusion d1 d2)
;;

### II. 2. Déséquilibre

Un ABR est dit équilibré si et seulement si $h = O(\log|A|)$.

Le déséquilibre d'un arbre $A = N(g, \_ , d)$ vaut $d(A) := h(g) - h(d)$.

Un arbre AVL est un arbre dans lequel le déséquilibre de chaque sous-arbre est égal à −1, 0 ou 1.

### II. 3. Parcours

Il existe trois grand types de parcours d'ABR : préfixe, infixe et suffixe, selon l'ordre dans lequel on visite les noeuds.

Le parcours préfixe visite d'abord la racine, puis le sous-arbre gauche, puis le sous-arbre droit.

Le parcours infixe visite d'abord le sous-arbre gauche, puis la racine, puis le sous-arbre droit. Il est particulièrement utile pour un ABR car il permet de parcourir les noeuds dans l'ordre croissant.

Le parcours suffixe visite d'abord le sous-arbre gauche, puis le sous-arbre droit, puis la racine.

**Complexité :** $O(|A|)$

In [ ]:
let rec prefixe f = function
| V -> ()
| N (g, x, d) -> f x ; prefixe g ; prefixe d
;;

let rec infixe f = function
| V -> ()
| N (g, x, d) -> infixe g ; f x ; infixe d
;;

let rec postfixe f = function
| V -> ()
| N (g, x, d) -> postfixe g ; postfixe d : f x
;;

(* Application : vérification du caractère ABR d'un arbre *)
let est_abr arbre =
  let rec infixe acc = function
  | V -> acc
  | N (g, x, d) -> infixe (x :: (infixe acc g)) d in

  let rec est_decroissante = function
  | [] | [_] -> true
  | a :: b :: l -> a >= b && est_decroissante b::l
  in

  est_decroissante (infixe [] arbre)
;;

---
## III. Table de hachage
Structure de donnée impérative efficace pour implémenter un dictionnaire. Elle améliore ainsi la structure d'arbre binaire de recherche.

Son implémentation telle que détaillée ici consiste en un `'a option array`. La fonction d'insertion fonctionne par **sondage linéaire** : elle examine les cases suivantes dans l'ordre en sélectionnant la première case vide. 

Une analyse de complexité amortie peut montrer que les opérations ont un coût constant en moyenne, pour un facteur de charge bien choisi.

**Complexité :** $O(1)$ en moyenne

In [ ]:
let est_present hashtbl hash el =
  let n = Array.length hashtbl in
  let i = ref (hash el) in
  let found = ref false in
  while !i < n && not !found do
    match hashtbl.(!i) with
    | None -> ()
    | Some a -> if a = el then found := true
  done ;
  !found
;;

let inserer hashtbl hash el =
  let i = ref (hash el) in
  let placed = ref false in
  while not !placed do
    match hashtbl.(!i) with
    | None -> hashtbl.(!i) <- Some el ; placed := true
    | Some _ -> i := !i + 1 mod Array.length hashtbl
  done
;;

let position hashtbl hash el =
  let n = Array.length hashtbl in
  let i = ref (hash el) in
  let pos = ref None in
  while !i < n && !pos <> None do
    match hashtbl.(!i) with
    | None -> ()
    | Some a -> if a = el then pos := Some !i
  done ;
  !pos
;;

let swap t i j =
  let mem = t.(i) in
  t.(i) <- t.(j) ;
  t.(j) <- mem
;;

let supprimer hashtbl hash el =
  let n = Array.length hashtbl in
  let pos = position hashtbl hash el in
  match pos with
  | None -> ()
  | Some p -> 
    hashtbl.(p) <- None ;

    let rec decaler i =
      (* 'decaler' suppose que hashtbl.(i-1 mod n) = None et que 0 <= i < n *)
      match hashtbl.(i) with
      | None -> ()
      | Some v ->
        if hash v <> i then begin
          swap hashtbl i (i - 1 mod n) ; 
          decaler (i + 1 mod n)
        end
    in
    decaler (p + 1 mod n)
;;

---
## IV. File de priorité et tas

Un tas est un arbre tournoi quasi-complet à gauche. Tournoi : chaque étiquette de chaque noeud est inférieure à celle de ses enfants.  

**Complexité** : $O(\log(|S|))$ dans le pire cas. 

In [ ]:
type 'a tas = {
  mutable taille : int ;
  contenu : (int * 'a) array
}

let creer_tas capacite elt =
  { taille = 0 ; contenu = Array.make capacite (0, elt) };;

let est_vide tas = tas.taille = 0;;

let echanger t i j =
  let cache = t.(i) in 
  t.(i) <- t.(j) ;
  t.(j) <- cache
;;

(* Percolation vers le haut, pour corriger l'absence de caractère tournoi de l'arbre lors d'un ajout *)
let rec remontee tab i =
  if i > 0 then begin
    let parent = (i + 1) / 2 - 1 in begin
        if tab.(i) < tab.(parent) then begin
          echanger tab i parent ;
          remontee tab parent
        end
    end
  end;;

(* Percolation vers le bas, pour s'assurer que la nouvelle racine soit bien mise à sa place lors d'une extraction*)
let rec descente tab courant taille = 
  (* Définition des futurs candidats *)
  let candidat = ref courant in 
  let fils_gauche = (2 * courant) + 1 in 
  let fils_droit = fils_gauche + 1 in

  (* Analyse du fils gauche *)
  if fils_gauche < taille then begin
    if tab.(fils_gauche) < tab.(!candidat) then
      candidat := fils_gauche ;
    
    (* Analyse du fils droit*)
    if fils_droit < taille then begin
      if tab.(fils_droit) < tab.(!candidat) then
        candidat := fils_droit
    end;
    
    (* En cas de changement, on réalise l'échange et on continue récursivement *)
    if !candidat <> courant then begin
      echanger tab courant !candidat;
      descente tab !candidat taille
    end
  end
;;

(* Ajout d'un élément dans le tas *)
let ajout tas elt priorite =
  if tas.taille >= Array.length tas.contenu then
    failwith "ajout : tas plein" ;
  tas.contenu.(tas.taille) <- (priorite, elt) ;
  remontee tas.contenu tas.taille;
  tas.taille <- tas.taille + 1
;;

(* Extraction de la racine du tas *)
let extraction tas =
  if tas.taille = 0 then
    failwith "extraction : tas vide" ;
  

  let prio, racine = tas.contenu.(0) in 
  tas.taille <- tas.taille - 1 ;

  (* On ajoute le dernier élément à la racine (pour conserver le caractère semi-complet) *)
  tas.contenu.(0) <- tas.contenu.(tas.taille) ;
  (* On percole vers le bas pour corriger le caractère tournoi *)
  descente tas.contenu 0 tas.taille ;
  (prio, racine)
;;

### Tri par tas
Toutes les percolations sont en temps logarithmique, donc le tri par tas est en $O(n \log n)$ dans tous les cas. C'est un tri en place, mais non stable. C'est le seul tri en place connu en $O(n \log n)$. (L'implémentation suivante trie par ordre décroissant, mais on peut modifier les fonctions pour obtenir l'ordre croissant.)

In [ ]:
let tri_par_tas t =
  let n = Array.length t in 
  (* On transforme le tableau en tas (tournoi) *)
  for i = 1 to n-1 do 
    remontee t i 
  done ;

  (* "Extraction" des éléments dans l'ordre *)
  for i = n - 1 downto 1 do 
    echanger t 0 i ; (* On prend l'élément à la racine (le plus petit) et on le met à la fin *)
    descente t 0 i (* On percole vers le bas pour conserver la structure tournoi sur [| 0 ; i-1 |] *)
  done
;;

---
## V. File (HP)

**Complexité :** $O(1)$

In [ ]:
type 'a queue = {tab : 'a array ; mutable deb : int ; mutable fin : int ; mutable vide : bool};;

let vide cap el_init = {tab = Array.make cap el_init ; deb = 0 ; fin = 0 ; vide = true};;

let est_vide q = q.vide;;

let insertion q el = 
  if q.fin mod Array.length q.tab = q.deb && (not q.vide) then failwith "queue pleine"
  else begin
    q.tab.(q.fin) <- el ;
    q.fin <- q.fin + 1 mod Array.length q.tab ;
    q.vide <- false
  end
;;

let suppression q el =
  if q.vide then failwith "queue vide"
  else begin
    q.deb <- q.deb + 1 mod Array.length q.tab ;
    if q.fin = q.deb then q.vide <- true
  end
;;

---
## VI. Parcours en profondeur de graphes et applications

**Complexité :** $O(|S| + |A|)$

### VI. 1. Parcours en profondeur

In [ ]:
let parcours_en_profondeur g = (* g est un tableau de listes contenants les voisins *)
  let n = Array.length g in 
  let etats = Array.make n 0 in 

  let rec traiter sommet = 
    if etats.(sommet) = 0 then begin
      etats.(sommet) <- 1 ;
      (* pré-traitement *)
      List.iter traiter g.(sommet) ;
      (* post-traitement *)
      etats.(sommet) <- 2
    end
  in 
  for s = 0 to n - 1 do 
    traiter s
  done
;;

### VI. 2. Tri topologique

In [ ]:
let tri_topologique g =
  let n = Array.length g in 
  let etats = Array.make n 0 in 
  let tri = ref [] in

  let rec traiter sommet = 
    if etats.(sommet) = 0 then begin 
      etats.(sommet) <- 1 ;
      List.iter traiter g.(sommet) ;
      tri := sommet :: !tri ;
    end 
  in 
  for i = 0 to n-1 do 
    traiter i
  done ;
  !tri
;;

### VI. 3. Recherche et construction d'un cycle

In [ ]:
let detection_de_cycle g =
  let n = Array.length g in 
  let etats = Array.make n 0 in 
  let cycle = ref false in

  let rec traiter sommet = 
    if !cycle || etats.(sommet) = 1 then
      cycle := true
    else
      if etats.(sommet) = 0 then begin
        etats.(sommet) <- 1 ;

        List.iter traiter g.(sommet) ;

        etats.(sommet) <- 2
      end
  in 
  for s = 0 to n - 1 do 
    traiter s
  done ;
  !cycle
;;

In [ ]:
let calcul_cycle g =
  let n = Array.length g in 
  let etats = Array.make n (0, -1) in (* contient des couples (etat, prédécesseur) *)
  let fin_cycle = ref (-1) in (* contient un sommet de début=fin de cycle, s'il existe *)

  let rec traiter sommet pred =
    if !fin_cycle = -1 then begin (* si un cycle a été détecté, on arrête tout *)
      if fst etats.(sommet) = 1 then fin_cycle := sommet (* on a détecté un cycle *)
      else 
        if fst etats.(sommet) = 0 then begin
          etats.(sommet) <- (1, pred) ;
          List.iter (fun v -> traiter v sommet) g.(sommet) ;
          etats.(sommet) <- (2, pred)
        end
    end
  in

  for s = 0 to n-1 do
    traiter s (-1)
  done ;

  if !fin_cycle = -1 then None
  else begin
    (* reconstruction du chemin *)
    let s = ref (snd etats.(!fin_cycle)) in
    let cycle = ref [snd etats.(!fin_cycle)] in
    while !s <> !fin_cycle do
      s := snd etats.(!s) ;
      cycle := !s :: !cycle
    done;
    Some !cycle
  end
;;

### VI. 4. Calcul des composantes connexes

In [ ]:
let composantes_connexes g = (* graphe non orienté *)
  let n = Array.length g in
  let vus = Array.make n false in
  let compo = ref [] in
  
  let rec visite sommet = 
    if not vus.(sommet) then begin
      vus.(sommet) <- true ;
      compo := sommet :: !compo ;
      List.iter visite g.(sommet)
    end
  in

  let composantes = ref [] in
  for i = 0 to n-1 do
    compo := [] ;
    visite i ;
    if !compo <> [] then composantes := !compo :: !composantes
  done;
  !composantes
;;

### VI. 5. États accessibles d'un automate

Voir le TP de l'épreuve d'algorithmique sur machine. Pas de difficulté particulière.

### VI. 6. Calcul des composantes fortement connexes (Algorithme de Kosoraju, HP)
Les composantes fortement connexes d'un graphe sont les classes d'équivalence de la relation de co-accessibilité. $p$ et $q$ sont dans la même composante connexe si et seulement si il existe un chemin de $p$ à $q$ et un chemin de $q$ à $p$.

L'idée de l'algorithme est de faire deux parcours. Le premier parcours en profondeur est semblable à un tri topologique et permet d'obtenir une liste des sommets correctement ordonnée. On réalise ensuite une transposition du graphe. Enfin, on parcourt le graphe transposé dans l'ordre énoncé par le premier parcours, en ajoutant au fur et à mesure les sommets à leur composante connexe, et en commençant une nouvelle composante à chaque fin du parcours.

**Complexité** : $O(|S| + |A|)$ (optimale)

In [ ]:
(* On construit une liste des sommets à l'aide d'un parcours en profondeur, comme pour un tri topologique *)
let parcours1 g =
  let n = Array.length g in 
  let a_traiter = Array.make n true in 
  let l = ref [] in
  
  let rec traiter sommet = 
    if a_traiter.(sommet) then begin
      a_traiter.(sommet) <- false ; 
      l := sommet :: !l ;
      List.iter traiter g.(sommet);
    end
  in 

  for s = 0 to n-1 do 
    traiter s
  done;
  !l
;;

(* On calcule le graphe transposé *)
let transpose g = 
  let n = Array.length g in 
  let gt = Array.make n [] in 

  for i = 0 to n-1 do 
    List.iter (fun j -> gt.(j) <- i :: gt.(j)) gt.(i)
  done;
  gt
;;

(* On effectue un parcours en profondeur en respectant l'ordre de la liste du premier parcours *)
let parcours2 g ordre = 
  let n = Array.length g in 
  let a_traiter = Array.make n true in 

  let rec traiter sommet composante = 
    if a_traiter.(sommet) then begin 
      a_traiter.(sommet) <- false ;
      List.iter (fun s -> traiter s composante) g.(sommet);
      composante := sommet :: !composante
    end
  in 
  let composantes = ref [] in 

  let ajout_composante sommet = 
    if a_traiter.(sommet) then begin 
      let composante = ref [] in 
        traiter sommet composante ;
        composantes := composante :: !composantes 
    end
  in

  List.iter ajout_composante ordre;
  !composantes
;;

(* Mise en forme *)
let kosaraju g = 
  let ordre = parcours1 g in 
  let gt = transpose g in 
  parcours2 gt ordre
;;

---
## VII. Plus court chemin dans un graphe

### VII. 1. Algorithme de Dijkstra
Permet d'obtenir le plus court chemin entre un sommet $s$ et tous les autres sommets d'un graphe pondéré positivement. 

**Complexité** : $O((|S| + |A|) \times \log|S|)$ pour une implémentation de la file de priorité avec un tas classique. En effet, le parcours est en $O(|S| + |A|)$, et l'insertion et la suppression dans un tas sont en $O(\log|S|)$.

(Pour un tas de Fibonacci, la complexité est en $O(|S| \times \log|S| + |A|)$.)

In [ ]:
let dijkstra graphe source dest = 
  let n = Array.length graphe in

  (* Initialisation de la file de priorité *)
  let tas = creer_tas n (source, -1) in
  ajout tas (source, -1) 0 ; (* le tas est constituté de couples (sommet, provenance) *)
  let predecesseur = Array.make n (-1) in (* -1 si jamais vu, prédécesseur[i] sinon *)

  let arriv = ref false in
  while (not (est_vide tas)) && (not !arriv) do 
    let dist_e, (e, provenance) = extraction tas in
    if e = dest then arriv := true
    else 
      if predecesseur.(e) = -1 then begin
        predecesseur.(e) <- provenance ;
        List.iteri (fun vois dist -> ajout tas (vois, e) (dist + dist_e)) graphe.(e)
      end
  done;

  if not !arriv then None
  else begin
    (* Récupération du chemin *)
    let chemin = ref [] in
    let sommet = ref dest in
    while !sommet <> source do
      chemin := !sommet :: !chemin ;
      sommet := predecesseur.(!sommet)
    done;
    Some !chemin
  end
;;

### VII. 2. Algorithme de Floyd-Warshall
Permet d'obtenir le plus court chemin entre tous les sommets d'un graphe orienté. Le graphe peut avoir des poids négatifs, mais ne doit pas avoir de cycle de poids négatif.

**Complexité** : $O(|S|^3)$

In [ ]:
let floyd_warshall graphe = 
  (* Initialisation *)
  let n = Array.length graphe in
  let w = Array.make_matrix n n Int.max_int in

  for i = 0 to n-1 do
    w.(i).(i) <- 0
  done;

  for i = 0 to n-1 do
    for j = 0 to n-1 do
      w.(i).(j) <- graphe.(i).(j)
    done
  done ;

  (* Itérations *)
  for k = 0 to n-1 do
    for i = 0 to n-1 do
      for j = 0 to n-1 do
        w.(i).(j) <- min w.(i).(j) (w.(i).(k) + w.(k).(j))
      done
    done
  done ;
  w
;;

---
## VIII. Arbres couvrants minimaux

On rappelle qu'un arbre est un graphe non-orienté connexe et acyclique. Ceci se caractérise de plusieurs autres façon rappelant l'algèbre linéaire. Les propositions suivantes sont équivalents :
- G est un arbre
- G est connexe et $Card A = Card S - 1$
- G est acyclique et $Card A = Card S - 1$
- G est connexe et ne le reste pas en supprimant une arête
- G est acyclique et ne le reste pas en ajoutant un arête

### VIII. 1. Algorithme de Prim (HP)
Le fonctionnement est le suivant :
- On choisit un sommet du graphe, et on considère $T_1$ contenant ce sommet et aucune arête.
- Pour passer de $T_n$ à $T_{n+1}$, on établit la liste des arêtes reliant un sommet de $T_n$ à un sommet hors de $T_n$, et on sélectionne l'arête de poids minimal et le sommet associé.
- Quand on a sélectionné tous les sommets, on a fini.

Son implémentation repose sur une file de priorité (`Heap`) pouvant être implémentée par un tas.

**Complexité :** $O(|S| + |A| \log(|A|))$. Le graphe étant supposé connexe, $|S| = O(|A|)$ et pour tout graphe, $|A| \leq |S|^2$, donc $\log(|A|) = \log(|S|)$, d'où une complexité en $O(|A| \log(|S|))$.

In [ ]:
let prim g =
  let n = Array.length g in
  let vus = Array.make n false in
  let arbre = ref [] in
  let tas = Heap.empty () in

  let ajout_sommet sommet = 
    (* marque le sommet comme vu et ajoute toutes les arêtes dans la file de priorité *)
    vus.(sommet) <- true ;
    List.iter 
      (fun (voisin, distance) -> 
        Heap.inser (distance, sommet, voisin) tas) g.(sommet)
  in

  let rec cherche_arete () = 
    (* extrait une arête menant à un sommet non visité *)
    let _, debut, fin = Heap.extract tas in (* poids, début et fin de l'arête *)
    if vus.(fin) then
      (* on connaît fin, donc on continue *)
      cherche_arete ()
    else
      (debut, fin)
  in

  ajoute_sommet 0 ;
  for i = 1 to n-1 do
    let debut, fin = cherche_arete () in
    arbre := (debut, fin) :: !arbre ;
    ajoute_sommet fin
  done ;
  !arbre
;;

### VIII. 2. Algorithme de Kruskal
Utilise à la fois une file de priorité et également une structure `Union-Find` :
- On part de chaque sommet étant sa propre composante connexe, sans arêtes.
- On énumère les arêtes par poids croissant, et dès qu'une arête relie deux composantes connexes distinctes, on sélectionne l'arête et on fusionne les composantes.

In [ ]:
let kruskal g =
  let n = Array.length g in
  let tas = Heap.empty () in
  let uf = UF.init n in
  let arbre = ref [] in

  let rec cherche_arete () =
    let _, debut, fin = Heap.extract tas in
    if UF.union uf debut fin then (* si  les composantes sont distinctes *)
      (debut, fin)
    else
      cherche_arete () (* sinon, on continue à chercher *)
  in

  for s = 0 to n-1 do
    List.iter (fun (v, d) -> Heap.insert (d, s, v) tas) ;
    g.(s)
  done ;

  (* On effectue n-1 fusions *)
  for i = 1 to n-1 do
    let debut, fin = cherche_arete () in
    arbre := (debut, fin) :: !arbre 
  done;
  !arbre
;;

---
## IX. Graphes bipartis et couplage maximal
Un graphe non-orienté $G = (S, A)$ est dit **biparti** s'il est possible de partager $S$ en deux sous-ensembles distincts tels que chaque arc de $A$ relie un sommet d'un sous-ensemble à un sommet de l'autre sous-ensemble. 

De manière formelle, $\exists U, V \subset S$ tels que $U \cap V = \emptyset$, $U \cup V = S$ et $A \subseteq U \times V$.

### IX. 1. Bi-colorabilité
Un graphe $G = (S, A)$ admet une $k$-coloration (pour $k \geq 2$) s'il existe une fonction de coloration $c$ de $S$ dans $\{1, \dots, k\}$ telle que pour tout arc $(u, v) \in A$, $c(u) \neq c(v)$. 

Un graphe est biparti si et seulement si il admet une 2-coloration, qui correspond à la fonction indicatrice de $U$ par exemple.

In [ ]:
let est_bicolorable graphe =
  let n = Array.length graphe in
  let couleurs = Array.make n (-1) in
  let possible = ref true in

  let rec visite sommet cl = 
  match couleurs.(sommet) with
  | -1 -> couleurs.(sommet) <- cl ; List.iter (fun s -> s (cl + 1 mod 2)) graphe.(sommet)
  | _ -> if cl <> couleurs.(sommet) then possible := false
  in

  for i = 0 to n-1 do
    visite i 0
  done;

  !possible
;;

### IX. 2. Couplage maximal

Représentations d'un graphe biparti :

In [ ]:
type biparti_1 = {
  aretes : int list array ;
  division : int
};;

type biparti_2 = {
  u : int list array ;
  v : int list array
};;

Un **couplage** est un sous-ensemble $M \subseteq A$ d'arêtes tel que deux arêtes distinctes n'ont pas de sommet en commun. De façon équivalente, $M$ est un couplage si et seulement si pour tout sommet $s \in S$, il existe au plus une arête de $M$ incidente à $s$.

Un sommet qui n'est pas extrémité d'une arête de $M$ est dit **libre**.

Un chemin $C$ de $M$ est dit **alternant** si il alterne entre des arêtes de $M$ et des arêtes de $A \setminus M$.

Un chemin **augmentant** est un chemin alternant dont les extrémités sont libres.

**Lemme :** Si $M$ est un couplage et $C$ un chemin augmentant, alors $M' := M \Delta C$ est un couplage avec $|M'| = |M| + 1$.

**Théorème de Berge :** Si $M_1$ et $M_2$ sont deux couplages tels que $|M_1| < |M_2|$, alors il existe un chemin augmentant pour $M_1$. Conséquence : un couplage $M$ est maximal si et seulement si il n'existe pas de chemin augmentant pour $M$.

In [ ]:
(* Renvoie le premier élément d'une liste qui vérifie p, et None s'il n'y en a pas *)
let rec existe p = function
| [] -> None
| a :: l -> if p a then Some a else existe p l
;;

let couplage_maximal g =
  let nu = Array.length g.u
  and nv = Array.length g.v in

  let vus = Array.make nu false in
  let couplage = Array.make nv None in (* les sommets sont initialement libres *)

  let rec visite_sommet sommet =
    if vus.(sommet) then false
    else begin
      vus.(sommet) <- true ;
      match existe bon_voisin g.u.(sommet) with
      | None -> false
      | Some voisin -> couplage.(voisin) <- Some sommet ; true
    end
  and
  bon_voisin voisin =
    match couplage.(voisin) with
    | None -> true
    | Some sommet -> visite_sommet sommet
  in
  for s = 0 to nu-1 do
    (* On cherche un chemin augmentant partant du sommet s*)
    for i = 0 to nu - 1 do
      vus.(i) <- false
    done ;
    ignore (visite_sommet s)
  done;
  couplage
;;

---
## X. Union Find (HP)

### X.1. Implémentation naïve

### X. 2. Implémentation améliorée

---
## XI. Backtracking

### XI. 1. Somme d'un sous-ensemble

### XI. 2. Problème des reines

---
## XII. Théorème de Kleene

### XII. 1. Rationnel $\implies$ reconnaissable
Voyons deux méthodes pour transformer une `regex` en automate.

### XII. 1. a. Automate de Thompson
On définit de façon inductive des "briques" reconnaissant les opérations de base d'une expression régulière, en utilisant abondement des $\varepsilon$-transitions (automate asynchrone). On peut alors synchroniser et déterminiser l'automate obtenu.

### XII. 1. b. Automate de Glushkov
- **Langage local :** Un langage $L$ est dit local s'il existe des ensembles $P, S \subseteq \Sigma$ et $F \subseteq \Sigma^2$ tels que tout mot est un mot de $L$ si et seulement si il commence par une lettre de $P$, finit par une lettre de $S$, et contient uniquement des facteurs de taille 2 dans $F$. On note alors $L = (V, P, F, S)$ où $V \subseteq \{\varepsilon\}$

L'intersection, l'union\*, la concaténation\*, et l'étoile de langages locaux sont locaux.

(\* : à condition que les alphabets soient disjoints)

- **Expression rationnelle linéaire :** Expression rationnelle dont chaque lettre apparaît au plus une fois.

Toute expression rationnelle linéaire dénote un langage local (immédiat par définition).

- **Automate local :** automate déterministe dont toutes les transitions étiquetées par la même lettre arrivent dans le même état. Il est dit standard si aucune transition n'arrive sur l'état initial.

Tout langage local est reconnu par un automate local standard.

On en déduit que tout langage décrit par une expression régulière linéaire est reconnaissble. On définit pour cela l'automate d'état initial $\varepsilon$, d'états finals $F_A = F \cup V$, et de transitions : $\forall a \in P, \delta(\varepsilon, a) = a$ et $\forall ab \in F, \delta(a, b) = b$.

On généralise à toutes les expressions linéaires en numérotant les lettres identiques.

### XII. 1. c. Algorithme de Berry-Sethi
- Linéariser l'expression régulière
- Faire un tableau avec les $V, P, F, S$
- Construire l'automate local la reconnaissant en utilisant la règle de construction ci-dessus
- Effacer la numérotation des lettres


## XII. 2. Reconnaissable $\implies$ rationnel
Voyons deux méthodes pour transformer un automate déterministe en `regex`

## XII. 2. a. Lemme d'Arden

## XII. 2. b. Méthode de Brzozowksi et McCluskey
On élimine des états en remplaçant les transitions par des `regex`. Méthode à suivre :
- On ajoute un état initial supplémentaire avec des $\varepsilon$-transitions vers chaque état de $I$
- On ajoute un état final supplémentaire avec des $\varepsilon$-transitions depuis chaque état de $F$
- Tant qu'il reste un état de $Q$ non éliminé, on fusionne les transitions et on réduit un état.